Представьте, что вы работаете аналитиком в очень крупной компании по доставке пиццы над приложением для курьеров (да, обычно в таких компаниях есть приложение для курьеров и отдельно приложение для потребителей).

У вас есть несколько ресторанов в разных частях города и целый штат курьеров. Но есть одна проблема — к вечеру скорость доставки падает из-за того, что курьеры уходят домой после рабочего дня, а количество заказов лишь растет. Это приводит к тому, что в момент пересмены наша доставка очень сильно проседает в эффективности.

Наши data scientist-ы придумали новый алгоритм, который позволяет курьерам запланировать свои последние заказы перед окончанием рабочего дня так, чтобы их маршрут доставки совпадал с маршрутом до дома. То есть, чтобы курьеры доставляли последние свои заказы за день как бы "по пути" домой.

Вы вместе с командой решили раскатить A/B тест на две равные группы курьеров. Часть курьеров использует старый алгоритм без опции "по пути", другие видят в своем приложении эту опцию и могут ее выбрать. Ваша задача – проанализировать данные эксперимента и помочь бизнесу принять решение о раскатке новой фичи на всех курьеров.

`Описание данных`

order_id - id заказа

delivery_time - время доставки в минутах

district - район доставки

experiment_group - экспериментальная группа

`ГИПОТЕЗЫ`

Н0 - разницы между временем доставкой в тестовой и контрольной группе есть

Н1 - разница есть)

In [6]:
import pandas as pd
import numpy as np

In [8]:
data = pd.read_csv('https://stepik.org/media/attachments/lesson/385916/experiment_lesson_4.csv')
data.head()

,order_id,delivery_time,district,experiment_group
0,3159,44.0,Purlieus,control
1,10426,51.0,Purlieus,control
2,9155,46.0,Purlieus,control
3,10699,34.0,Purlieus,control
4,10061,43.0,Purlieus,control


Посчитайте количество наблюдений в тестовой и контрольной группе. На сколько по модулю отличается количество наблюдений в тесте и контроле?

In [35]:
test_count = data.query('experiment_group == "test"').value_counts().sum()
test_count

10104

In [39]:
control_count = data.query('experiment_group == "control"').value_counts().sum()
control_count

10092

In [43]:
dif = test_count - control_count
dif

12

Теперь давайте изучим наши распределения. Проверим выполнение условий на нормальность и посчитаем стандартное отклонение в тесте и в контроле.

Обязательно сделайте семплирование перед применением критерия при проверке на нормальность (и для теста, и для контроля), так как на больших выборках вы можете получать неверные результаты:

experiment[experiment['experiment_group'] == 'test']['delivery_time'].sample(1000, random_state=17)
Почему тест может дать здесь неправильный ответ? Загляните в подсказки - в первом пункте мы постарались осветить этот момент подробнее :)

In [50]:
from scipy.stats import shapiro
shapiro(data[data['experiment_group']=='test']['delivery_time'].sample(1000, random_state=17))

ShapiroResult(statistic=0.9975429485383064, pvalue=0.13902241238486684)

In [52]:
shapiro(data[data['experiment_group']=='control']['delivery_time'].sample(1000, random_state=17))

ShapiroResult(statistic=0.9974716381542054, pvalue=0.12392829880543467)

In [62]:
std_test = round(data.query('experiment_group == "test"').delivery_time.std(), 3)
std_test

9.883

In [64]:
std_control = round(data.query('experiment_group == "control"').delivery_time.std(), 3)
std_control

9.99

`Вывод:` Так как p-value для двух выборок через критерий Шапиро-Уилка больше 0.05, то мы не отвергаем 0-ю гипотезу о том, что распределение нормальное. Так же нашли среднеквадратичное отклонение для тестовой выборки = 9.88 и для контрольной выборки = 9.99

Пришло время сравнить средние в тесте и контроле. Внимание: здесь уже не нужно пользоваться методом .sample() — он нам нужен был только для теста Шапиро-Уилка, остальные тесты надо проводить на полных данных.



In [68]:
from scipy.stats import ttest_ind
ttest_ind(data.query('experiment_group == "test"').delivery_time, 
          data.query('experiment_group == "control"').delivery_time)

TtestResult(statistic=-43.03582663183714, pvalue=0.0, df=20194.0)

`Вывод:` Мы сравнили средние двух групп. P-value = 0, значит мы отвергаем Н0, значение статистики равно -43

На сколько процентов изменилось среднее время доставки в тестовой группе по сравнению с контрольной (округлите до сотых)?

In [72]:
test = data.query('experiment_group == "test"')
test_mean = test.delivery_time.mean()
test_mean

39.04681314330958

In [76]:
control_mean = data.query('experiment_group == "control"').delivery_time.mean()
control_mean

45.065101070154576

In [80]:
percent = (control_mean - test_mean) / control_mean * 100
round(percent, 2)

13.35

`Ответ:` На 13,35% изменилось время доставки в лучшую сторону